# 🌟 Pipeline 2: Image Enhancement and Restoration
## Advanced Image Processing for Quality Improvement

### 🎯 Problem Statement:
**"Given a low-quality image, enhance it using various image processing techniques"**

### 📋 Solution Approach:
1. **Image Analysis** → Identify quality issues
2. **Noise Reduction** → Remove different types of noise
3. **Contrast Enhancement** → Improve visibility
4. **Sharpening** → Enhance details
5. **Color Correction** → Fix color issues
6. **Final Optimization** → Combine best techniques

### 🔧 Techniques Used:
- Histogram analysis and equalization
- Various denoising filters
- Contrast stretching and CLAHE
- Unsharp masking and sharpening
- White balancing algorithms
- Frequency domain filtering

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage

def show_results(images, titles, rows=2, cols=3, figsize=(15, 10)):
    """Display multiple images in a grid"""
    fig, axes = plt.subplots(rows, cols, figsize=figsize)
    axes = axes.flatten() if rows * cols > 1 else [axes]
    
    for i, (img, title) in enumerate(zip(images, titles)):
        if i >= len(axes):
            break
        ax = axes[i]
        
        if len(img.shape) == 3:
            ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        else:
            ax.imshow(img, cmap='gray')
        ax.set_title(title, fontweight='bold')
        ax.axis('off')
    
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')
    
    plt.tight_layout()
    plt.show()

def create_degraded_image():
    """Create a test image with various quality issues"""
    # Start with a simple test pattern
    img = np.zeros((300, 300, 3), dtype=np.uint8)
    
    # Add geometric patterns
    cv2.rectangle(img, (50, 50), (150, 150), (180, 180, 180), -1)
    cv2.circle(img, (200, 80), 40, (120, 120, 120), -1)
    cv2.rectangle(img, (80, 180), (180, 250), (160, 160, 160), -1)
    
    # Add some text-like features
    for i in range(5):
        cv2.line(img, (200, 150 + i*10), (270, 150 + i*10), (100, 100, 100), 2)
    
    # Simulate various degradations
    
    # 1. Add Gaussian noise
    noise = np.random.normal(0, 25, img.shape).astype(np.int16)
    img = np.clip(img.astype(np.int16) + noise, 0, 255).astype(np.uint8)
    
    # 2. Add salt and pepper noise
    mask = np.random.random(img.shape[:2])
    img[mask < 0.01] = 0     # Salt noise
    img[mask > 0.99] = 255   # Pepper noise
    
    # 3. Reduce contrast
    img = np.clip(img * 0.6 + 50, 0, 255).astype(np.uint8)
    
    # 4. Add blur (motion blur simulation)
    kernel = np.zeros((9, 9))
    kernel[4, :] = 1/9
    img = cv2.filter2D(img, -1, kernel)
    
    # 5. Color cast (simulating poor white balance)
    img[:, :, 0] = np.clip(img[:, :, 0] * 1.2, 0, 255)  # Blue cast
    
    return img

def analyze_image_quality(img):
    """Analyze image quality metrics"""
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if len(img.shape) == 3 else img
    
    # Calculate various quality metrics
    mean_brightness = np.mean(gray)
    contrast = np.std(gray)
    dynamic_range = gray.max() - gray.min()
    
    # Estimate noise using Laplacian variance
    laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()
    
    # Edge density
    edges = cv2.Canny(gray, 50, 150)
    edge_density = np.sum(edges > 0) / edges.size
    
    return {
        'brightness': mean_brightness,
        'contrast': contrast,
        'dynamic_range': dynamic_range,
        'noise_estimate': laplacian_var,
        'edge_density': edge_density
    }

print("🌟 Image Enhancement and Restoration Pipeline Ready!")

## 📖 Step 1: Load and Analyze Image Quality

In [ ]:
# Load or create test image
try:
    img = cv2.imread('D:/FPT_Material/Sem 4/CPV301/Source for PE/Image/low_contrast_image2.png')
    if img is None:
        raise FileNotFoundError("Image not found")
    print("✅ Loaded image from file")
except:
    print("⚠️ Creating degraded test image...")
    img = create_degraded_image()

# Convert to different formats
rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Analyze image quality
quality_metrics = analyze_image_quality(img)

print(f"📊 Image Quality Analysis:")
print(f"Shape: {img.shape}")
print(f"Brightness: {quality_metrics['brightness']:.1f} (0-255)")
print(f"Contrast (std): {quality_metrics['contrast']:.1f}")
print(f"Dynamic Range: {quality_metrics['dynamic_range']}")
print(f"Noise Estimate: {quality_metrics['noise_estimate']:.1f}")
print(f"Edge Density: {quality_metrics['edge_density']:.3f}")

# Show histogram
plt.figure(figsize=(15, 4))

plt.subplot(1, 3, 1)
plt.imshow(rgb_img)
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.hist(gray_img.ravel(), 256, [0, 256], alpha=0.7)
plt.title('Histogram')
plt.xlabel('Pixel Intensity')
plt.ylabel('Frequency')

plt.subplot(1, 3, 3)
# Show color channel histograms
colors = ['red', 'green', 'blue']
for i, color in enumerate(colors):
    hist = cv2.calcHist([img], [i], None, [256], [0, 256])
    plt.plot(hist, color=color, alpha=0.7, label=f'{color.upper()} channel')
plt.title('Color Channel Histograms')
plt.xlabel('Pixel Intensity')
plt.ylabel('Frequency')
plt.legend()

plt.tight_layout()
plt.show()

# Store original for comparison
original_img = img.copy()
original_gray = gray_img.copy()

## 🔧 Step 2: Noise Reduction

In [ ]:
# ========================================
# 2.1 Different Denoising Techniques
# ========================================

# Gaussian blur for general noise
gaussian_denoised = cv2.GaussianBlur(img, (5, 5), 0)

# Median filter for salt-and-pepper noise
median_denoised = cv2.medianBlur(img, 5)

# Bilateral filter for edge-preserving denoising
bilateral_denoised = cv2.bilateralFilter(img, 9, 75, 75)

# Non-local means denoising (advanced)
nlm_denoised = cv2.fastNlMeansDenoisingColored(img, None, 10, 10, 7, 21)

# Morphological opening for noise removal
kernel = np.ones((3, 3), np.uint8)
morphological_denoised = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)

# ========================================
# 2.2 Custom Wiener-like Filter
# ========================================
def wiener_filter(img, noise_variance=100):
    """Simple Wiener-like filter implementation"""
    # Convert to frequency domain
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    f_transform = np.fft.fft2(img_gray)
    f_shift = np.fft.fftshift(f_transform)
    
    # Create Wiener filter
    rows, cols = img_gray.shape
    power_spectrum = np.abs(f_shift) ** 2
    wiener_filter = power_spectrum / (power_spectrum + noise_variance)
    
    # Apply filter
    filtered = f_shift * wiener_filter
    f_ishift = np.fft.ifftshift(filtered)
    img_filtered = np.fft.ifft2(f_ishift)
    img_filtered = np.abs(img_filtered)
    
    # Convert back to color
    result = img.copy()
    for i in range(3):
        result[:, :, i] = cv2.normalize(img_filtered, None, 0, 255, cv2.NORM_MINMAX)
    
    return result.astype(np.uint8)

wiener_denoised = wiener_filter(img)

# Analyze noise reduction effectiveness
print("🔧 Noise Reduction Analysis:")
print(f"Original noise estimate: {analyze_image_quality(img)['noise_estimate']:.1f}")
print(f"Gaussian denoised: {analyze_image_quality(gaussian_denoised)['noise_estimate']:.1f}")
print(f"Median denoised: {analyze_image_quality(median_denoised)['noise_estimate']:.1f}")
print(f"Bilateral denoised: {analyze_image_quality(bilateral_denoised)['noise_estimate']:.1f}")
print(f"NLM denoised: {analyze_image_quality(nlm_denoised)['noise_estimate']:.1f}")

show_results(
    [cv2.cvtColor(img, cv2.COLOR_BGR2RGB), 
     cv2.cvtColor(gaussian_denoised, cv2.COLOR_BGR2RGB),
     cv2.cvtColor(median_denoised, cv2.COLOR_BGR2RGB),
     cv2.cvtColor(bilateral_denoised, cv2.COLOR_BGR2RGB),
     cv2.cvtColor(nlm_denoised, cv2.COLOR_BGR2RGB),
     cv2.cvtColor(wiener_denoised, cv2.COLOR_BGR2RGB)],
    ['Original', 'Gaussian', 'Median', 'Bilateral', 'NLM', 'Wiener-like'],
    rows=2, cols=3
)

# Choose best denoising result
denoised_img = bilateral_denoised  # Often provides good balance
print("✅ Selected bilateral filter for further processing")

## 📈 Step 3: Contrast Enhancement

In [ ]:
# ========================================
# 3.1 Global Contrast Enhancement
# ========================================

# Convert to grayscale for some operations
denoised_gray = cv2.cvtColor(denoised_img, cv2.COLOR_BGR2GRAY)

# Histogram Equalization
hist_eq = cv2.equalizeHist(denoised_gray)

# Contrast Stretching (Normalization)
contrast_stretched = cv2.normalize(denoised_gray, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)

# CLAHE (Contrast Limited Adaptive Histogram Equalization)
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
clahe_enhanced = clahe.apply(denoised_gray)

# ========================================
# 3.2 Gamma Correction
# ========================================
def gamma_correction(img, gamma=1.2):
    """Apply gamma correction"""
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(img, table)

gamma_corrected = gamma_correction(denoised_gray, gamma=0.8)  # Brighten

# ========================================
# 3.3 Local Contrast Enhancement
# ========================================
def local_contrast_enhancement(img, kernel_size=9):
    """Enhance local contrast using local statistics"""
    # Calculate local mean and standard deviation
    mean = cv2.blur(img.astype(np.float32), (kernel_size, kernel_size))
    sqr_img = img.astype(np.float32) ** 2
    sqr_mean = cv2.blur(sqr_img, (kernel_size, kernel_size))
    std = np.sqrt(sqr_mean - mean ** 2)
    
    # Local contrast enhancement
    enhanced = img.astype(np.float32)
    mask = std > 0
    enhanced[mask] = mean[mask] + 2 * (enhanced[mask] - mean[mask]) * (std[mask] / (std[mask] + 1))
    
    return np.clip(enhanced, 0, 255).astype(np.uint8)

local_enhanced = local_contrast_enhancement(denoised_gray)

# ========================================
# 3.4 Adaptive Contrast Enhancement
# ========================================
def adaptive_contrast(img, alpha=1.5, beta=50):
    """Adaptive contrast enhancement"""
    # Calculate local statistics
    blurred = cv2.GaussianBlur(img.astype(np.float32), (15, 15), 0)
    enhanced = alpha * (img.astype(np.float32) - blurred) + blurred + beta
    return np.clip(enhanced, 0, 255).astype(np.uint8)

adaptive_enhanced = adaptive_contrast(denoised_gray)

# Analyze contrast improvement
print("📈 Contrast Enhancement Analysis:")
print(f"Original contrast (std): {np.std(denoised_gray):.1f}")
print(f"Histogram equalized: {np.std(hist_eq):.1f}")
print(f"Contrast stretched: {np.std(contrast_stretched):.1f}")
print(f"CLAHE enhanced: {np.std(clahe_enhanced):.1f}")
print(f"Gamma corrected: {np.std(gamma_corrected):.1f}")
print(f"Local enhanced: {np.std(local_enhanced):.1f}")
print(f"Adaptive enhanced: {np.std(adaptive_enhanced):.1f}")

show_results(
    [denoised_gray, hist_eq, contrast_stretched, clahe_enhanced, gamma_corrected, local_enhanced],
    ['Denoised', 'Hist Equalized', 'Contrast Stretched', 'CLAHE', 'Gamma Corrected', 'Local Enhanced'],
    rows=2, cols=3
)

# Choose best contrast enhancement
contrast_enhanced = clahe_enhanced  # Often provides good results
print("✅ Selected CLAHE for contrast enhancement")

## ⚡ Step 4: Sharpening and Detail Enhancement

In [ ]:
# ========================================
# 4.1 Traditional Sharpening Kernels
# ========================================

# Basic sharpening kernel
sharpen_kernel = np.array([[-1, -1, -1],
                          [-1,  9, -1],
                          [-1, -1, -1]])
kernel_sharpened = cv2.filter2D(contrast_enhanced, -1, sharpen_kernel)

# Laplacian sharpening
laplacian = cv2.Laplacian(contrast_enhanced, cv2.CV_64F)
laplacian_sharpened = np.clip(contrast_enhanced.astype(np.float64) - 0.3 * laplacian, 0, 255).astype(np.uint8)

# ========================================
# 4.2 Unsharp Masking
# ========================================
def unsharp_mask(img, blur_size=5, strength=1.5, threshold=0):
    """Apply unsharp masking for sharpening"""
    # Create blurred version
    blurred = cv2.GaussianBlur(img, (blur_size, blur_size), 0)
    
    # Calculate mask
    mask = img.astype(np.float32) - blurred.astype(np.float32)
    
    # Apply threshold
    if threshold > 0:
        low_contrast_mask = np.absolute(mask) < threshold
        mask[low_contrast_mask] = 0
    
    # Apply mask
    sharpened = img.astype(np.float32) + strength * mask
    
    return np.clip(sharpened, 0, 255).astype(np.uint8)

unsharp_sharpened = unsharp_mask(contrast_enhanced, blur_size=3, strength=1.2)

# ========================================
# 4.3 High-Boost Filtering
# ========================================
def high_boost_filter(img, amplification=1.5):
    """Apply high-boost filtering"""
    # Low-pass filter
    blurred = cv2.GaussianBlur(img, (5, 5), 0)
    
    # High-frequency component
    high_freq = img.astype(np.float32) - blurred.astype(np.float32)
    
    # High-boost filter
    boosted = img.astype(np.float32) + amplification * high_freq
    
    return np.clip(boosted, 0, 255).astype(np.uint8)

high_boost_sharpened = high_boost_filter(contrast_enhanced, amplification=1.3)

# ========================================
# 4.4 Frequency Domain Sharpening
# ========================================
def frequency_domain_sharpen(img):
    """Sharpen using frequency domain high-pass filter"""
    # FFT
    f_transform = np.fft.fft2(img)
    f_shift = np.fft.fftshift(f_transform)
    
    # Create high-pass filter
    rows, cols = img.shape
    crow, ccol = rows // 2, cols // 2
    
    # Butterworth high-pass filter
    def butterworth_hp(d0, n=2):
        r, c = np.mgrid[0:rows, 0:cols]
        d = np.sqrt((r - crow)**2 + (c - ccol)**2)
        return 1 / (1 + (d0 / (d + 1e-8))**(2*n))
    
    hp_filter = butterworth_hp(30)
    
    # Apply filter
    filtered = f_shift * hp_filter
    
    # IFFT
    f_ishift = np.fft.ifftshift(filtered)
    img_filtered = np.fft.ifft2(f_ishift)
    img_filtered = np.abs(img_filtered)
    
    # Combine with original
    result = img.astype(np.float32) + 0.5 * img_filtered
    
    return np.clip(result, 0, 255).astype(np.uint8)

freq_sharpened = frequency_domain_sharpen(contrast_enhanced)

# Analyze sharpening effectiveness
def calculate_sharpness(img):
    """Calculate image sharpness using Laplacian variance"""
    return cv2.Laplacian(img, cv2.CV_64F).var()

print("⚡ Sharpening Analysis:")
print(f"Original sharpness: {calculate_sharpness(contrast_enhanced):.1f}")
print(f"Kernel sharpened: {calculate_sharpness(kernel_sharpened):.1f}")
print(f"Laplacian sharpened: {calculate_sharpness(laplacian_sharpened):.1f}")
print(f"Unsharp masked: {calculate_sharpness(unsharp_sharpened):.1f}")
print(f"High-boost filtered: {calculate_sharpness(high_boost_sharpened):.1f}")
print(f"Frequency sharpened: {calculate_sharpness(freq_sharpened):.1f}")

show_results(
    [contrast_enhanced, kernel_sharpened, laplacian_sharpened, 
     unsharp_sharpened, high_boost_sharpened, freq_sharpened],
    ['Contrast Enhanced', 'Kernel Sharpen', 'Laplacian Sharpen', 
     'Unsharp Mask', 'High-Boost', 'Frequency Sharpen'],
    rows=2, cols=3
)

# Choose best sharpening result
sharpened_img = unsharp_sharpened  # Often provides good balance
print("✅ Selected unsharp masking for detail enhancement")

## 🎨 Step 5: Color Correction and White Balance

In [ ]:
# ========================================
# 5.1 Apply enhancements to color image
# ========================================

# Apply the same processing pipeline to color image
# Start with denoised color image
enhanced_color = denoised_img.copy()

# Apply CLAHE to each channel separately
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
lab = cv2.cvtColor(enhanced_color, cv2.COLOR_BGR2LAB)
lab[:, :, 0] = clahe.apply(lab[:, :, 0])  # Apply to L channel only
enhanced_color = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

# Apply unsharp masking to each channel
for i in range(3):
    enhanced_color[:, :, i] = unsharp_mask(enhanced_color[:, :, i], blur_size=3, strength=1.1)

# ========================================
# 5.2 White Balance Correction
# ========================================

def gray_world_white_balance(img):
    """Gray World white balancing"""
    avg_b, avg_g, avg_r = np.mean(img, axis=(0, 1))
    avg = (avg_b + avg_g + avg_r) / 3
    
    scale_b = avg / avg_b if avg_b > 0 else 1
    scale_g = avg / avg_g if avg_g > 0 else 1
    scale_r = avg / avg_r if avg_r > 0 else 1
    
    balanced = enhanced_color.astype(np.float32)
    balanced[:, :, 0] *= scale_b
    balanced[:, :, 1] *= scale_g
    balanced[:, :, 2] *= scale_r
    
    return np.clip(balanced, 0, 255).astype(np.uint8)

def white_patch_white_balance(img, percentile=99):
    """White Patch white balancing"""
    max_b = np.percentile(img[:, :, 0], percentile)
    max_g = np.percentile(img[:, :, 1], percentile)
    max_r = np.percentile(img[:, :, 2], percentile)
    
    max_val = 250.0
    scale_b = max_val / max_b if max_b > 0 else 1
    scale_g = max_val / max_g if max_g > 0 else 1
    scale_r = max_val / max_r if max_r > 0 else 1
    
    balanced = enhanced_color.astype(np.float32)
    balanced[:, :, 0] = np.clip(balanced[:, :, 0] * scale_b, 0, 255)
    balanced[:, :, 1] = np.clip(balanced[:, :, 1] * scale_g, 0, 255)
    balanced[:, :, 2] = np.clip(balanced[:, :, 2] * scale_r, 0, 255)
    
    return balanced.astype(np.uint8)

gray_world_balanced = gray_world_white_balance(enhanced_color)
white_patch_balanced = white_patch_white_balance(enhanced_color)

# ========================================
# 5.3 Color Saturation Enhancement
# ========================================
def enhance_saturation(img, saturation_scale=1.3):
    """Enhance color saturation"""
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV).astype(np.float32)
    hsv[:, :, 1] *= saturation_scale  # Increase saturation
    hsv[:, :, 1] = np.clip(hsv[:, :, 1], 0, 255)
    return cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2BGR)

saturation_enhanced = enhance_saturation(gray_world_balanced)

# ========================================
# 5.4 Vibrance Enhancement (selective saturation)
# ========================================
def enhance_vibrance(img, vibrance=30):
    """Enhance vibrance (selective saturation boost)"""
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV).astype(np.float32)
    
    # Get current saturation
    s = hsv[:, :, 1] / 255.0
    
    # Calculate vibrance boost (less saturated pixels get more boost)
    boost = vibrance * (1 - s) / 100.0
    
    # Apply boost
    hsv[:, :, 1] = np.clip(hsv[:, :, 1] + boost * 255, 0, 255)
    
    return cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2BGR)

vibrance_enhanced = enhance_vibrance(gray_world_balanced)

print("🎨 Color Analysis:")
def analyze_color_balance(img, name):
    avg_b, avg_g, avg_r = np.mean(img, axis=(0, 1))
    print(f"{name}: B={avg_b:.1f}, G={avg_g:.1f}, R={avg_r:.1f}")

analyze_color_balance(enhanced_color, "Enhanced")
analyze_color_balance(gray_world_balanced, "Gray World")
analyze_color_balance(white_patch_balanced, "White Patch")

show_results(
    [cv2.cvtColor(enhanced_color, cv2.COLOR_BGR2RGB),
     cv2.cvtColor(gray_world_balanced, cv2.COLOR_BGR2RGB),
     cv2.cvtColor(white_patch_balanced, cv2.COLOR_BGR2RGB),
     cv2.cvtColor(saturation_enhanced, cv2.COLOR_BGR2RGB),
     cv2.cvtColor(vibrance_enhanced, cv2.COLOR_BGR2RGB)],
    ['Enhanced Color', 'Gray World WB', 'White Patch WB', 'Saturation Enhanced', 'Vibrance Enhanced'],
    rows=2, cols=3
)

# Choose final color correction
final_color = vibrance_enhanced
print("✅ Selected vibrance enhancement for final color")

## 🎯 Step 6: Final Optimization and Comparison

In [ ]:
# ========================================
# 6.1 Final Refinements
# ========================================

# Apply a subtle final sharpening
final_enhanced = final_color.copy()
for i in range(3):
    final_enhanced[:, :, i] = unsharp_mask(final_enhanced[:, :, i], blur_size=5, strength=0.3)

# Subtle noise reduction on the final result
final_enhanced = cv2.bilateralFilter(final_enhanced, 5, 10, 10)

# ========================================
# 6.2 Quality Assessment
# ========================================
print("🎯 Final Quality Assessment:")
print("=" * 50)

original_quality = analyze_image_quality(original_img)
final_quality = analyze_image_quality(final_enhanced)

improvements = {
    'brightness': final_quality['brightness'] - original_quality['brightness'],
    'contrast': final_quality['contrast'] - original_quality['contrast'],
    'dynamic_range': final_quality['dynamic_range'] - original_quality['dynamic_range'],
    'noise_reduction': original_quality['noise_estimate'] - final_quality['noise_estimate'],
    'edge_density': final_quality['edge_density'] - original_quality['edge_density']
}

print("Quality Metrics Comparison:")
print(f"Brightness: {original_quality['brightness']:.1f} → {final_quality['brightness']:.1f} ({improvements['brightness']:+.1f})")
print(f"Contrast: {original_quality['contrast']:.1f} → {final_quality['contrast']:.1f} ({improvements['contrast']:+.1f})")
print(f"Dynamic Range: {original_quality['dynamic_range']} → {final_quality['dynamic_range']} ({improvements['dynamic_range']:+.0f})")
print(f"Noise Level: {original_quality['noise_estimate']:.1f} → {final_quality['noise_estimate']:.1f} ({-improvements['noise_reduction']:+.1f})")
print(f"Edge Density: {original_quality['edge_density']:.3f} → {final_quality['edge_density']:.3f} ({improvements['edge_density']:+.3f})")

# ========================================
# 6.3 Side-by-Side Comparison
# ========================================
comparison_imgs = [
    cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB),
    cv2.cvtColor(denoised_img, cv2.COLOR_BGR2RGB),
    contrast_enhanced,
    sharpened_img,
    cv2.cvtColor(gray_world_balanced, cv2.COLOR_BGR2RGB),
    cv2.cvtColor(final_enhanced, cv2.COLOR_BGR2RGB)
]

comparison_titles = [
    'Original Image',
    'Denoised',
    'Contrast Enhanced',
    'Sharpened',
    'Color Corrected',
    'Final Enhanced'
]

show_results(comparison_imgs, comparison_titles, rows=2, cols=3, figsize=(18, 12))

# ========================================
# 6.4 Enhancement Summary
# ========================================
print(f"\n📊 Enhancement Pipeline Summary:")
print("=" * 50)
print("✅ Processing Steps Applied:")
print("1. Noise Reduction - Bilateral filtering")
print("2. Contrast Enhancement - CLAHE")
print("3. Detail Enhancement - Unsharp masking")
print("4. Color Correction - Gray world white balance")
print("5. Saturation Enhancement - Vibrance boost")
print("6. Final Refinement - Subtle sharpening and denoising")

print(f"\n📈 Key Improvements:")
if improvements['contrast'] > 0:
    print(f"✓ Contrast improved by {improvements['contrast']:.1f}")
if improvements['noise_reduction'] > 0:
    print(f"✓ Noise reduced by {improvements['noise_reduction']:.1f}")
if improvements['dynamic_range'] > 0:
    print(f"✓ Dynamic range increased by {improvements['dynamic_range']:.0f}")
if improvements['edge_density'] > 0:
    print(f"✓ Edge definition improved by {improvements['edge_density']:.3f}")

print(f"\n🎉 Image Enhancement Completed Successfully!")
print(f"📚 This pipeline demonstrates:")
print(f"   - Comprehensive noise reduction techniques")
print(f"   - Advanced contrast enhancement methods")
print(f"   - Multiple sharpening approaches")
print(f"   - Color correction and white balancing")
print(f"   - Quality assessment and comparison")